# Guessing Earlier Bounding Boxes

It seems that COTS are sometimes visible prior to the first appearance of the bounding box.  In this notebook we will try to exploit that and create more annotations in earlier frames.

**Hypothesis:** we can enlarge our training data set by adding earlier bounding boxes prior to the first detected box for each COTS

**Approach:**
1. Add detections to earlier frames that have a detection in a subsequent frame as follows:
- identify frames that have less detections than the next one
- exclude any candidate boxes if the box has an overlap with another box in the previous frame
- exclude the image "margins" to account for that some boxes may not have been visible in the frame
- shift the bounding box by the average translation of any other matched boxes
2. Compare some examples frames right before and right after a detection
3. Save the results into a new training set

**Results:**
This approach results in a small increase in number of annotations and frames with at least 1 annotation. Some of these new annotations are not very good fit (see examples below) but some seem pretty good.  I have not tried in a training pipeline but if you do please leave a comment how the new training data set did.

- Previous number of bounding boxes:  11898
- New number of boxes:  12045
- Number of boxes increase:  147

- Previous number of frames with boxes:  4919
- New number of frames with boxes:  4963
- Number of frames with boxes increase:  44

Please let me know if you find any bugs! ;)

Under Development:
1. Use "optical flow" techniques to determine where the boxes may have come from better, fit tighter boxes, and project boxes into margins if possible. 
2. Add even more annotations by looking at even earlier frames potentially using "reverse" tracking

# Please UPVOTE if you find this useful!  Thank you!

In [ ]:
import cv2 as cv
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math

from PIL import Image, ImageDraw
import tensorflow as tf

import os
import ast
import sys
import time

import greatbarrierreef

# Parameters

In [ ]:
IMAGE_DIM = (1280,720)

EXCLUDE_MARGIN = 0.02 #we will not be adding boxes in 2% of the image width or height

## Read Data

In [ ]:
def get_bbox(annots):
    bboxes = [list(annot.values()) for annot in annots]
    return bboxes

def read_data():
    df_train = pd.read_csv('../input/tensorflow-great-barrier-reef/train.csv')
    df_train['img_path'] = os.path.join('../input/tensorflow-great-barrier-reef/train_images')+"/video_"+df_train.video_id.astype(str)+"/"+df_train.video_frame.astype(str)+".jpg"
    df_train['annotations'] = df_train['annotations'].apply(lambda x: ast.literal_eval(x))
    df_train['bboxes'] = df_train['annotations'].apply(lambda x: get_bbox(x))
    df_train['Number_bbox'] = df_train['annotations'].apply(lambda x:len(x)) 
    return df_train

In [ ]:
df_train = read_data()

In [ ]:
df_train.sample(5).head()

## Add New Bounding Boxes to Previous Frames

In [ ]:
#shift next annotations to previous frame
df_shift = df_train.shift(-1).rename(columns={'annotations':'annotations_n1',
                                             'Number_bbox':'Number_bbox_n1',
                                             'img_path':'img_path_n1'})
df_lagged = pd.concat([df_train, df_shift], axis=1)

#identify frames that have less annotations then the next one
#these are the candidates for adding earlier annotations
df_first_frames = df_lagged[df_lagged.Number_bbox < df_lagged.Number_bbox_n1]

In [ ]:
def intersects(rectangle_a, rectangle_b):
    '''Checks for intersection of two rectangles specified as [(x1,y1),(x2,y2)]'''
    if(rectangle_a[1][0]<rectangle_b[0][0] or rectangle_a[1][1]<rectangle_b[0][1]):
        return False
    elif(rectangle_a[0][0]>rectangle_b[1][0] or rectangle_a[0][1]>rectangle_b[1][1]):
        return False
    else:
        return True
        
def new_bboxes(prev_bboxes, next_bboxes):
    '''Returns the bounding boxes that are deemed new in the next frame by checking 
    the centers of the bounding box in the next frame are not contained in
    one of the previous frame bounding boxes.'''
    new_bbs =[]
    delta_xs = [0]
    delta_ys = [0]
    delta_ws = [0]
    delta_hs = [0]
    for bb in next_bboxes:
        found = False
        for prev_bb in prev_bboxes:
            if intersects([(bb['x'],bb['y']),(bb['x'] + bb['width'],bb['y'] + bb['height'])],
                         [(prev_bb['x'], prev_bb['y']), (prev_bb['x'] + prev_bb['width'], 
                                                         prev_bb['y'] + prev_bb['height'])]
                         ):
                delta_xs.append(bb['x']-prev_bb['x'])
                delta_ys.append(bb['y']-prev_bb['y'])
                delta_ws.append(bb['width']-prev_bb['width'])
                delta_hs.append(bb['height']-prev_bb['height'])
                found = True
                break
        if found == False:
            #exclude margins
            if (bb['x'] > IMAGE_DIM[0]*EXCLUDE_MARGIN) & \
            (bb['x'] < (IMAGE_DIM[0]-IMAGE_DIM[0]*EXCLUDE_MARGIN)) & \
            (bb['y'] > IMAGE_DIM[1]*EXCLUDE_MARGIN) & \
            (bb['y'] < (IMAGE_DIM[1]-IMAGE_DIM[1]*EXCLUDE_MARGIN)):
                new_bb = {'x': bb['x'], 'y': bb['y'], 'width':bb['width'], 'height':bb['height']}
                new_bbs.append(new_bb)
                
    #adjust bounding boxes for avergage drift
    for b in new_bbs:        
        delta_x_avg = sum(delta_xs)/len(delta_xs)
        delta_y_avg = sum(delta_ys)/len(delta_ys)
        delta_w_avg = sum(delta_ws)/len(delta_ws)
        delta_h_avg = sum(delta_hs)/len(delta_hs)
        b['x'] = round(b['x'] + delta_x_avg)
        b['y'] = round(b['y'] + delta_y_avg)
        #adjusting width/height does not seem to improve bounding box fit
        #b['width'] = b['width'] + delta_w_avg
        #b['height'] = b['height'] + delta_h_avg
               
    return new_bbs

In [ ]:
df_first_frames['new_annotations'] = df_first_frames.apply(lambda x: 
                                                            new_bboxes(x['annotations'],
                                                                      x['annotations_n1']),
                                                          axis=1)

## See some examples of new annotations

In [ ]:
def viz_new_boxes(prev_path, next_path, prev_annots, next_annots, new_annots):    
    #previuos frame
    print(prev_path)
    img = Image.open(prev_path)
    
    for box in prev_annots:
        shape = [box['x'], box['y'], box['x']+box['width'], box['y']+box['height']]
        ImageDraw.Draw(img).rectangle(shape, outline ="red", width=3)

    for box in next_annots:
        shape = [box['x'], box['y'], box['x']+box['width'], box['y']+box['height']]
        ImageDraw.Draw(img).rectangle(shape, outline ="cyan", width=3)

    
    for box in new_annots:
        shape = [box['x'], box['y'], box['x']+box['width'], box['y']+box['height']]
        ImageDraw.Draw(img).rectangle(shape, outline ="yellow", width=3)

    display(img)    
    
    #next frame
    print(next_path)
    img = Image.open(next_path)
    
    for box in next_annots:
        shape = [box['x'], box['y'], box['x']+box['width'], box['y']+box['height']]
        ImageDraw.Draw(img).rectangle(shape, outline ="red", width=3)

    #for box in new_annots:
    #    shape = [box['x'], box['y'], box['x']+box['width'], box['y']+box['height']]
    #    ImageDraw.Draw(img).rectangle(shape, outline ="orange", width=3)
        
    display(img)

Note that we are able to add a few new bounding boxes (yellow) although the fit is not great due to camera movement.  Future development will be to leverage optical flow alogrithm or similar to improve the fit.

In [ ]:
for index, row in df_first_frames.sample(10, random_state=12).iterrows():
    viz_new_boxes(row.img_path,
                  row.img_path_n1,
                  row.annotations,
                  row.annotations_n1,
                  row.new_annotations)

# Merge the new and existing annotations

In [ ]:
df_train = df_train.join(df_first_frames['new_annotations'])

In [ ]:
df_train['new_annotations'] = df_train['new_annotations'].fillna('[]')

In [ ]:
def combine_annotations(a1, a2):
    if type(a2) == str:
        a2 = []
    return a1 + a2

df_train['annotations_combined'] = df_train.apply(lambda x: combine_annotations(x['annotations'],
                                                            x['new_annotations']),
                                                            axis=1)

In [ ]:
df_train.annotations_combined.iloc[34]

# Count how many new annotations we have

In [ ]:
df_train['New_number_bbox'] = df_train['annotations_combined'].apply(lambda x: len(x))

In [ ]:
df_train[df_train['New_number_bbox'] > df_train['Number_bbox']]

In [ ]:
prev_box_count = df_train['Number_bbox'].sum()
curr_box_count = df_train['New_number_bbox'].sum()
prev_frames_with_box_count = df_train[df_train.Number_bbox >0].shape[0]
curr_frames_with_box_count = df_train[df_train.New_number_bbox >0].shape[0]
print("Previous number of bounding boxes: ", prev_box_count)
print("New number of boxes: ", curr_box_count)
print("Number of boxes increase: ", curr_box_count-prev_box_count)
print("Previous number of frames with boxes: ", prev_frames_with_box_count)
print("New number of frames with boxes: ", curr_frames_with_box_count)
print("Number of frames with boxes increase: ", curr_frames_with_box_count-prev_frames_with_box_count)

# Create a new train file

In [ ]:
cols = pd.read_csv('../input/tensorflow-great-barrier-reef/train.csv').columns

In [ ]:
df_new = df_train.copy()
df_new['annotations'] = df_train['annotations_combined']

In [ ]:
df_new[cols].to_csv('/kaggle/working/more_annotations_train.csv', index=False)

In [ ]:
pd.read_csv('/kaggle/working/more_annotations_train.csv').head()